In [1]:
import tensorflow as tf

In [2]:
# Create tf dataset from a list
daily_sales_numbers = [21, 22, -108, 31, -1, 32, 34,31]

tf_dataset = tf.data.Dataset.from_tensor_slices(daily_sales_numbers)
tf_dataset

<TensorSliceDataset shapes: (), types: tf.int32>

In [3]:
for sales in tf_dataset:
    print(sales.numpy())

21
22
-108
31
-1
32
34
31


In [4]:
# also
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

21
22
-108
31
-1
32
34
31


In [5]:
# Iterate through first n elements in tf dataset
for sales in tf_dataset.take(3):
    print(sales.numpy())

21
22
-108


In [6]:
# Filter sales numbers that are < 0
tf_dataset = tf_dataset.filter(lambda x: x>0)
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

21
22
31
32
34
31


In [7]:
# Convert sales numbers from USA dollars ($) to Indian Rupees (INR) Assuming 1->75 conversation rate
tf_dataset = tf_dataset.map(lambda x: x*75)
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

1575
1650
2325
2400
2550
2325


In [8]:
# batch of size 2
for sales in tf_dataset.batch(2):
    print(sales.numpy())

[1575 1650]
[2325 2400]
[2550 2325]


In [9]:
# all above operation in one, this is tensorflow input pipeline
tf_dataset= tf.data.Dataset.from_tensor_slices(daily_sales_numbers)
tf_dataset= tf_dataset.filter(lambda x: x>0).map(lambda y: y*75).shuffle(2).batch(2)

for sales in tf_dataset.as_numpy_iterator():
    print(sales)

[1575 2325]
[2400 2550]
[2325 1650]


### Images

In [10]:
# list of images
images_ds= tf.data.Dataset.list_files('datasets/*/*', shuffle=False)

for file in images_ds.take(3):
    print(file.numpy())

b'datasets\\cat\\00000001_000.jpg'
b'datasets\\cat\\00000001_005.jpg'
b'datasets\\cat\\00000001_008.jpg'


In [11]:
images_ds = images_ds.shuffle(200)
for file in images_ds.take(3):
    print(file.numpy())

b'datasets\\cat\\00000001_024.jpg'
b'datasets\\cat\\00000001_011.jpg'
b'datasets\\cat\\00000001_027.jpg'


In [12]:
class_name=['cat', 'dog']

In [13]:
image_count= len(images_ds)
image_count

30

In [14]:
# declaring train size
train_size= int(image_count*0.8)

# making training size 80%
train_ds= images_ds.take(train_size)

# skip 1st 80% of data to make test size 20%
test_ds= images_ds.skip(train_size)

In [15]:
print(len(train_ds))
print(len(test_ds))

24
6


In [16]:
# takeing label name from file path; this is demo for 1 file
s= 'datasets\\dog\\dog.4019.jpg'
s.split('\\')[-2]

'dog'

In [17]:
def get_label(file_path):
    import os
    # parts represent full path of file
    parts = tf.strings.split(file_path, os.path.sep)
    # it split and pick label from file path
    return parts[-2]

In [18]:
def process_image(file_path):
    label = get_label(file_path)
    img = tf.io.read_file(file_path) # load the raw data from the file as a string
    img = tf.image.decode_jpeg(img) # decode jpeg image
    img = tf.image.resize(img, [128, 128]) # resize image
    return img, label

In [19]:
img, label = process_image("datasets\\dog\\dog.4019.jpg")
img.numpy()[:2]

array([[[ 58.293213,  56.447388,  74.97864 ],
        [ 65.70776 ,  52.732788,  73.10315 ],
        [ 73.403564,  47.86194 ,  68.36182 ],
        [ 72.305786,  36.79016 ,  57.026733],
        [ 62.270386,  32.234375,  40.234375],
        [ 59.016235,  32.96216 ,  41.96216 ],
        [ 54.004272,  32.785522,  41.606567],
        [ 52.765747,  35.687622,  43.609497],
        [ 42.21045 ,  31.55017 ,  54.36267 ],
        [ 70.38916 ,  68.748535,  93.15442 ],
        [ 82.4375  ,  94.015625, 121.984375],
        [ 83.60071 , 107.47742 , 138.03906 ],
        [ 86.      , 115.      , 145.      ],
        [ 91.234375, 118.46875 , 144.82544 ],
        [ 90.71326 , 117.85388 , 134.01013 ],
        [ 75.140625, 102.140625,  96.953125],
        [ 83.42786 , 109.52161 , 100.38098 ],
        [ 95.330444, 122.330444, 113.611694],
        [100.43884 , 131.9701  , 119.20447 ],
        [101.31531 , 134.65906 , 120.90906 ],
        [105.111084, 135.97131 , 122.67273 ],
        [108.39636 , 136.92188 , 1

In [20]:
train_ds = train_ds.map(process_image)
test_ds = test_ds.map(process_image)

In [21]:
for image, label in train_ds.take(1):
    print("****",image)
    print("****",label)

**** tf.Tensor(
[[[ 73.831696  212.90982   238.37076  ]
  [ 67.796875  210.26562   236.64844  ]
  [ 62.492188  210.57812   234.88281  ]
  ...
  [ 93.04297   217.04297   242.4336   ]
  [ 96.256805  216.2568    240.874    ]
  [ 97.07779   213.07779   238.07779  ]]

 [[ 79.8125    217.8125    243.8125   ]
  [ 73.74307   214.11719   241.18448  ]
  [ 69.5809    214.5664    240.04688  ]
  ...
  [ 83.24548   208.98767   233.62943  ]
  [ 74.72922   196.31516   220.24786  ]
  [ 91.38861   210.48337   234.48337  ]]

 [[ 81.578094  217.5781    243.5781   ]
  [ 73.89401   211.0151    239.30807  ]
  [ 74.01895   212.73438   240.33203  ]
  ...
  [ 85.922455  217.31308   240.00839  ]
  [ 82.74835   208.36554   230.98273  ]
  [ 93.18582   212.85992   236.85992  ]]

 ...

 [[  3.3576965 187.73883   215.1529   ]
  [  5.7427063 190.44974   217.8638   ]
  [  9.978149  194.68518   222.68518  ]
  ...
  [ 12.34317   197.98944   225.77402  ]
  [  9.809723  197.80972   224.80972  ]
  [  5.1579285 194.        2

In [22]:
# scaling images
def scale(image, label):
    return image/255, label

In [25]:
train_ds = train_ds.map(scale)

In [26]:
for image, label in train_ds.take(5):
    print("****Image: ",image.numpy()[0][0])
    print("****Label: ",label.numpy())

****Image:  [0.00229509 0.00190011 0.00191465]
****Label:  b'cat'
****Image:  [0.00203232 0.00201694 0.00213997]
****Label:  b'dog'


UnknownError: NewRandomAccessFile failed to Create/Open: datasets\flower_photos\daisy : Access is denied.
; Input/output error
	 [[{{node ReadFile}}]] [Op:IteratorGetNext]

In [27]:
for image, label in train_ds.take(10):
    print("****Image: ",image.numpy()[0][0])
    print("****Label: ",label.numpy())

****Image:  [0.00273656 0.00236173 0.00260101]
****Label:  b'cat'
****Image:  [0.00273849 0.00332715 0.003334  ]
****Label:  b'dog'
****Image:  [0.00139033 0.0012343  0.00095224]
****Label:  b'dog'
****Image:  [0.00226404 0.00229336 0.00214942]
****Label:  b'cat'
****Image:  [0.00012357 0.00015433 0.00013895]
****Label:  b'cat'
****Image:  [0.000885   0.0012172  0.00158262]
****Label:  b'cat'


UnknownError: NewRandomAccessFile failed to Create/Open: datasets\flower_photos\tulips : Access is denied.
; Input/output error
	 [[{{node ReadFile}}]] [Op:IteratorGetNext]